In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic, NMF, KNNWithMeans, BaselineOnly
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise.accuracy import rmse, mae
import numpy as np

df = pd.read_csv('assignment/activity.csv')
df.head(1)

,user_id,liked_user_id,like_type,timestamp
0,462863,790124,0,2023-01-01 00:00:05.512765


#### Surprise library expects data in acertain schema:

In [2]:
df = df.rename(columns={'liked_user_id': 'item_id', 'like_type': 'rating'})
df.drop(columns=['timestamp'], inplace=True)
df.head()

,user_id,item_id,rating
0,462863,790124,0
1,717799,502110,1
2,288738,951824,0
3,106589,953301,0
4,338182,970712,0


### Check for sparsity:

In [3]:
# === SPARSITY ANALYSIS ===
print("\nSPARSITY ANALYSIS")
total_possible = df['user_id'].nunique() * df['item_id'].nunique()
sparsity = (1 - len(df) / total_possible) * 100
print(f"Matrix sparsity: {sparsity:.2f}%")
print(f"Density: {(100-sparsity):.4f}%")

if sparsity > 99.5:
    print("⚠️  EXTREMELY SPARSE DATA - Major issue!")
elif sparsity > 95:
    print("⚠️  Very sparse data - This hurts CF performance")

# === RATING DISTRIBUTION ===
print("\nRATING DISTRIBUTION")
rating_counts = df['rating'].value_counts().sort_index()
print(rating_counts)

unique_ratings = df['rating'].nunique()
print(f"Number of unique rating values: {unique_ratings}")

if unique_ratings == 2:
    print("⚠️  BINARY DATA DETECTED - Consider classification approach!")
elif unique_ratings < 10:
    print("⚠️  Limited rating scale - Consider ordinal approaches")

# === USER/ITEM RATING PATTERNS ===
print("\nUSER/ITEM RATING PATTERNS")
user_rating_counts = df['user_id'].value_counts()
item_rating_counts = df['item_id'].value_counts()

print(f"Users with only 1 rating: {(user_rating_counts == 1).sum()}")
print(f"Items with only 1 rating: {(item_rating_counts == 1).sum()}")
print(f"Median ratings per user: {user_rating_counts.median()}")
print(f"Median ratings per item: {item_rating_counts.median()}")

cold_users = (user_rating_counts == 1).sum() / len(user_rating_counts) * 100
cold_items = (item_rating_counts == 1).sum() / len(item_rating_counts) * 100

print(f"Cold start users: {cold_users:.1f}%")
print(f"Cold start items: {cold_items:.1f}%")


SPARSITY ANALYSIS
Matrix sparsity: 93.07%
Density: 6.9337%

RATING DISTRIBUTION
rating
0    1014887
1     425598
2       8312
Name: count, dtype: int64
Number of unique rating values: 3
⚠️  Limited rating scale - Consider ordinal approaches

USER/ITEM RATING PATTERNS
Users with only 1 rating: 218
Items with only 1 rating: 0
Median ratings per user: 180.0
Median ratings per item: 290.0
Cold start users: 5.2%
Cold start items: 0.0%


In [4]:
df['rating'].value_counts().sort_index()

rating
0    1014887
1     425598
2       8312
Name: count, dtype: int64

Given the provided data, we can consider three main
  strategies, each with its own strengths and weaknesses.

  1. Popularity-Based Recommender


  This is the most straightforward approach. The core idea is to recommend the users who are most popular across the entire platform,
  regardless of who the target user is.


   * How it works:
       1. Calculate a "popularity score" for every user. This score would be the total number of positive interactions (like_type 1 for
          "like" and 2 for "match") they have received.
       2. To generate recommendations for a specific user_id, you would take the list of all users, rank them by this popularity score in
          descending order.
       3. Finally, you would filter out any users the target user has already interacted with (liked, disliked, or matched) and return the
          top k users from the ranked list.
   * Pros:
       * Simple & Fast: Easy to implement and computationally inexpensive.
       * Good Baseline: Provides a solid starting point and is effective at recommending users who are generally considered attractive on
         the platform.
       * Solves Cold Start: It can generate recommendations for any user, even brand new ones with no interaction history.
   * Cons:
       * Not Personalized: Every user receives recommendations from the same pool of popular users. It doesn't adapt to individual
         preferences.
       * Popularity Bias: It can create a "rich get richer" effect, where popular users become even more visible, and less popular users are
         never discovered.

  2. Content-Based Filtering

  This approach uses the attributes of the users to recommend others with similar characteristics to those the target user has liked in the
  past.


   * How it works:
       1. Create User Profiles: For each user, create a profile vector based on their attributes in users.csv (e.g., gender, age, city, and
          even vectorized text from about_me using techniques like TF-IDF).
       2. Build Target User Profile: Analyze the profiles of users that the target user has positively interacted with (liked/matched).
          Combine these profiles to create an "ideal" profile that represents the target user's preferences.
       3. Find Similar Users: Calculate the similarity (e.g., using cosine similarity) between the target user's ideal profile and the
          profiles of all other candidate users.
       4. Recommend the top k most similar users.
   * Pros:
       * Highly Personalized: Recommendations are tailored to a user's specific tastes.
       * Explainable: You can easily explain why a recommendation was made (e.g., "Because you liked other users from London who are in
         their 30s").
       * No Cold Start for Items: It can recommend new users as long as they have a complete profile.
   * Cons:
       * User Cold Start: It cannot generate recommendations for new users who haven't interacted with anyone yet.
       * Filter Bubble: It may over-specialize and only recommend users who are very similar to past choices, limiting discovery of new
         types of profiles.
       * Requires Feature Engineering: The quality of recommendations heavily depends on how well you represent the user profiles from the
         available data.

  3. Collaborative Filtering

  This method makes recommendations based on the interactions of other users. The underlying assumption is that if two users have liked
  similar people in the past, they are likely to like similar people in the future.


   * How it works:
       1. Build a User-Interaction Matrix: Create a matrix where rows are users and columns are the users they've interacted with. The values
          would be the like_type.
       2. Find Similar Users: Using this matrix, find users who are "similar" to the target user by finding users who have liked/disliked the
          same set of people.
       3. Generate Recommendations: Recommend users that these similar users have liked, but which the target user has not yet seen.
       * Advanced methods like Matrix Factorization (SVD)** can be used to discover latent (hidden) features in the interaction data for more
         powerful predictions.
   * Pros:
       * Finds Surprising Connections: It can uncover non-obvious recommendations that content-based filtering would miss, as it's not limited
         by user attributes.
       * No Feature Engineering: It learns directly from user behavior, so you don't need to manually create user profiles.
   * Cons:
       * User/Item Cold Start: It struggles with both new users (no interaction history) and new users who have received no interactions.
       * Data Sparsity: If the platform is large and users have only interacted with a tiny fraction of other users, it can be difficult to
         find meaningful overlaps and the quality of recommendations suffers.
       * Computationally Expensive: Can be slow and memory-intensive, especially with large datasets.

In [5]:
reader = Reader(rating_scale=(0,2))

data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Collaborative Filtering Benchmarking

In [10]:
def benchmark_collaborative_filtering(data=data):
    """
    Comprehensive approach to improve collaborative filtering performance
    """

    # === 1. BASELINE COMPARISON ===
    print("=== BASELINE MODELS ===")
    
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
    
    # Simple baselines
    baseline_models = {
        'GlobalMean': BaselineOnly(bsl_options={'method': 'als'}),
        'SVD_Default': SVD(),
        'NMF_Default': NMF(),
        'KNN_Basic': KNNBasic(),
        'KNN_WithMeans': KNNWithMeans()
    }
    
    baseline_results = {}
    for name, model in baseline_models.items():
        model.fit(trainset)
        predictions = model.test(testset)
        rmse_score = rmse(predictions, verbose=False)
        mae_score = mae(predictions, verbose=False)
        baseline_results[name] = {'rmse': rmse_score, 'mae': mae_score}
        print(f"{name}: RMSE={rmse_score:.4f}, MAE={mae_score:.4f}")
    
    # === 2. HYPERPARAMETER TUNING ===
    print("\n=== HYPERPARAMETER TUNING ===")
    
    # SVD Grid Search
    print("Tuning SVD...")
    param_grid_svd = {
        'n_factors': [50, 100, 150],
        'n_epochs': [20, 30, 40],
        'lr_all': [0.002, 0.005, 0.01],
        'reg_all': [0.02, 0.05, 0.1]
    }
    
    gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
    gs_svd.fit(data)
    
    best_svd = gs_svd.best_estimator['rmse']
    print(f"Best SVD RMSE: {gs_svd.best_score['rmse']:.4f}")
    print(f"Best SVD params: {gs_svd.best_params['rmse']}")
    
    # NMF Grid Search  
    print("\nTuning NMF...")
    param_grid_nmf = {
        'n_factors': [50, 100, 150],
        'n_epochs': [20, 30, 40],
        'reg_pu': [0.06, 0.1, 0.15],
        'reg_qi': [0.06, 0.1, 0.15]
    }
    
    gs_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
    gs_nmf.fit(data)
    
    best_nmf = gs_nmf.best_estimator['rmse']
    print(f"Best NMF RMSE: {gs_nmf.best_score['rmse']:.4f}")
    print(f"Best NMF params: {gs_nmf.best_params['rmse']}")
    
    # KNN Grid Search
    print("\nTuning KNN...")
    param_grid_knn = {
        'k': [20, 30, 40, 50],
        'sim_options': {
            'name': ['pearson', 'msd'],
            'user_based': [True, False]
        }
    }
    
    gs_knn = GridSearchCV(KNNWithMeans, param_grid_knn, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
    gs_knn.fit(data)
    
    best_knn = gs_knn.best_estimator['rmse']
    print(f"Best KNN RMSE: {gs_knn.best_score['rmse']:.4f}")
    print(f"Best KNN params: {gs_knn.best_params['rmse']}")
    
    # === 3. FINAL COMPARISON ===
    print("\n=== FINAL COMPARISON ===")
    
    best_models = {
        'Best_SVD': best_svd,
        'Best_NMF': best_nmf, 
        'Best_KNN': best_knn
    }
    
    final_results = {}
    for name, model in best_models.items():
        model.fit(trainset)
        predictions = model.test(testset)
        rmse_score = rmse(predictions, verbose=False)
        mae_score = mae(predictions, verbose=False)
        final_results[name] = {'rmse': rmse_score, 'mae': mae_score, 'model': model}
        print(f"{name}: RMSE={rmse_score:.4f}, MAE={mae_score:.4f}")
    
    # Return best performing model
    best_model_name = min(final_results.keys(), key=lambda x: final_results[x]['rmse'])
    print(f"\nBest overall model: {best_model_name}")
    
    return final_results[best_model_name]['model'], final_results

result = benchmark_collaborative_filtering()

=== BASELINE MODELS ===
Estimating biases using als...
GlobalMean: RMSE=0.3736, MAE=0.2646
SVD_Default: RMSE=0.3758, MAE=0.2642
NMF_Default: RMSE=0.3778, MAE=0.2502
Computing the msd similarity matrix...
Done computing similarity matrix.
KNN_Basic: RMSE=0.3771, MAE=0.2603
Computing the msd similarity matrix...
Done computing similarity matrix.
KNN_WithMeans: RMSE=0.3777, MAE=0.2643

=== HYPERPARAMETER TUNING ===
Tuning SVD...
Best SVD RMSE: 0.3745
Best SVD params: {'n_factors': 50, 'n_epochs': 40, 'lr_all': 0.002, 'reg_all': 0.05}

Tuning NMF...
Best NMF RMSE: 0.3783
Best NMF params: {'n_factors': 50, 'n_epochs': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}

Tuning KNN...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...


/Users/sam/Documents/codes/inner_circle/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing

In [11]:
result

(<surprise.prediction_algorithms.matrix_factorization.SVD at 0x1060dc0b0>,
 {'Best_SVD': {'rmse': 0.37384830026043453,
   'mae': 0.2663610668695889,
   'model': <surprise.prediction_algorithms.matrix_factorization.SVD at 0x1060dc0b0>},
  'Best_NMF': {'rmse': 0.3775389658844272,
   'mae': 0.24977965958254525,
   'model': <surprise.prediction_algorithms.matrix_factorization.NMF at 0x11ce5ecc0>},
  'Best_KNN': {'rmse': 0.3751176686331623,
   'mae': 0.26323911359958424,
   'model': <surprise.prediction_algorithms.knns.KNNWithMeans at 0x1067c21b0>}})

### Aproaches to try:
- More aggressive filtering: Remove users/items with very few ratings
- Remove duplicates, keeping last rating
- Deep learning approaches (neural collaborative filtering)
- Ensemble methods
- Implicit feedback models (BPR, WARP)
- Add regularization and dropout

In [19]:
# Remove users/items with very few ratings
user_counts = df['user_id'].value_counts()
item_counts = df['item_id'].value_counts()

# More aggressive filtering
min_ratings = 250
valid_users = user_counts[user_counts >= min_ratings].index
valid_items = item_counts[item_counts >= min_ratings].index

df_clean = df[
    (df['user_id'].isin(valid_users)) & 
    (df['item_id'].isin(valid_items))
].copy()

# Remove duplicates, keeping last rating
df_clean = df_clean.drop_duplicates(['user_id', 'item_id'], keep='last')

print(f"Original: {len(df)} ratings")
print(f"Cleaned: {len(df_clean)} ratings")

if len(df_clean) < len(df) * 0.5:
    print("⚠️  Lost too much data in cleaning!")
    df_clean = df  # Revert to original

Original: 1448797 ratings
Cleaned: 1276645 ratings


In [21]:
reader = Reader(rating_scale=(0,2))

data = Dataset.load_from_df(df_clean[['user_id', 'item_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

results = benchmark_collaborative_filtering()

=== BASELINE MODELS ===
Estimating biases using als...
GlobalMean: RMSE=0.3736, MAE=0.2646
SVD_Default: RMSE=0.3758, MAE=0.2642
NMF_Default: RMSE=0.3778, MAE=0.2502
Computing the msd similarity matrix...
Done computing similarity matrix.
KNN_Basic: RMSE=0.3771, MAE=0.2603
Computing the msd similarity matrix...
Done computing similarity matrix.
KNN_WithMeans: RMSE=0.3777, MAE=0.2643

=== HYPERPARAMETER TUNING ===
Tuning SVD...
Best SVD RMSE: 0.3745
Best SVD params: {'n_factors': 50, 'n_epochs': 40, 'lr_all': 0.002, 'reg_all': 0.05}

Tuning NMF...
Best NMF RMSE: 0.3783
Best NMF params: {'n_factors': 50, 'n_epochs': 20, 'reg_pu': 0.06, 'reg_qi': 0.06}

Tuning KNN...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity ma

In [22]:
results

(<surprise.prediction_algorithms.matrix_factorization.SVD at 0x322c64680>,
 {'Best_SVD': {'rmse': 0.3738270261925852,
   'mae': 0.2663290632999272,
   'model': <surprise.prediction_algorithms.matrix_factorization.SVD at 0x322c64680>},
  'Best_NMF': {'rmse': 0.37753657543021096,
   'mae': 0.24978083922512823,
   'model': <surprise.prediction_algorithms.matrix_factorization.NMF at 0x11daab890>},
  'Best_KNN': {'rmse': 0.3751176686331623,
   'mae': 0.26323911359958424,
   'model': <surprise.prediction_algorithms.knns.KNNWithMeans at 0x303e82db0>}})

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from surprise import Dataset as SurpriseDataset, Reader, SVD, NMF, KNNWithMeans
from surprise.model_selection import train_test_split as surprise_split
from surprise.accuracy import rmse, mae

class NCFDataset(Dataset):
    """Custom dataset for Neural Collaborative Filtering"""
    def __init__(self, users, items, ratings):
        self.users = torch.LongTensor(users)
        self.items = torch.LongTensor(items)
        self.ratings = torch.FloatTensor(ratings)
    
    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

class NeuralCollaborativeFiltering(nn.Module):
    """Neural Collaborative Filtering model in PyTorch"""
    def __init__(self, n_users, n_items, embedding_dim=50, hidden_units=[128, 64], 
                 dropout_rate=0.3, l2_reg=0.01):
        super(NeuralCollaborativeFiltering, self).__init__()
        
        self.n_users = n_users
        self.n_items = n_items
        self.embedding_dim = embedding_dim
        
        # Embedding layers
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)
        
        # Apply L2 regularization to embeddings
        nn.init.normal_(self.user_embedding.weight, std=0.01)
        nn.init.normal_(self.item_embedding.weight, std=0.01)
        
        # Hidden layers
        layers = []
        input_dim = embedding_dim * 2  # Concatenated embeddings
        
        for hidden_dim in hidden_units:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            input_dim = hidden_dim
        
        # Output layer
        layers.append(nn.Linear(input_dim, 1))
        layers.append(nn.Sigmoid())  # For 0-1 rating range
        
        self.mlp = nn.Sequential(*layers)
        
        # L2 regularization parameter
        self.l2_reg = l2_reg
    
    def forward(self, users, items):
        # Get embeddings
        user_emb = self.user_embedding(users)
        item_emb = self.item_embedding(items)
        
        # Concatenate embeddings
        concat_emb = torch.cat([user_emb, item_emb], dim=1)
        
        # Pass through MLP
        output = self.mlp(concat_emb)
        
        return output.squeeze()
    
    def get_l2_loss(self):
        """Calculate L2 regularization loss"""
        l2_loss = 0
        for param in self.parameters():
            l2_loss += torch.norm(param) ** 2
        return self.l2_reg * l2_loss

class AdvancedCollaborativeFiltering:
    """
    Advanced collaborative filtering techniques for challenging datasets
    """
    
    def __init__(self, df, rating_scale=(0, 2)):
        self.df = df.copy()
        self.rating_scale = rating_scale
        self.user_encoder = LabelEncoder()
        self.item_encoder = LabelEncoder()
        self.device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
        print(f"Using device: {self.device}")
        self.prepare_data()
    
    def prepare_data(self):
        """Prepare data for advanced techniques"""
        # Remove duplicates
        self.df = self.df.drop_duplicates(['user_id', 'item_id'], keep='last')
        
        # Encode users and items to continuous integers
        self.df['user_encoded'] = self.user_encoder.fit_transform(self.df['user_id'])
        self.df['item_encoded'] = self.item_encoder.fit_transform(self.df['item_id'])
        
        self.n_users = self.df['user_encoded'].nunique()
        self.n_items = self.df['item_encoded'].nunique()
        
        print(f"Prepared data: {len(self.df)} ratings, {self.n_users} users, {self.n_items} items")
    
    def neural_collaborative_filtering(self, embedding_dim=50, hidden_units=[128, 64], 
                                     dropout_rate=0.3, l2_reg=0.01, epochs=50, 
                                     batch_size=256, learning_rate=0.001):
        """
        Neural Collaborative Filtering (NCF) implementation with PyTorch
        """
        print("\n=== NEURAL COLLABORATIVE FILTERING (PyTorch) ===")
        
        # Split data
        train_df, test_df = train_test_split(self.df, test_size=0.2, random_state=42)
        
        # Create datasets
        train_dataset = NCFDataset(
            train_df['user_encoded'].values,
            train_df['item_encoded'].values,
            train_df['rating'].values
        )
        
        test_dataset = NCFDataset(
            test_df['user_encoded'].values,
            test_df['item_encoded'].values,
            test_df['rating'].values
        )
        
        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        # Initialize model
        model = NeuralCollaborativeFiltering(
            n_users=self.n_users,
            n_items=self.n_items,
            embedding_dim=embedding_dim,
            hidden_units=hidden_units,
            dropout_rate=dropout_rate,
            l2_reg=l2_reg
        ).to(self.device)
        
        print(f"NCF Model Architecture:")
        print(model)
        print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
        
        # Loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        # Training loop
        model.train()
        train_losses = []
        
        for epoch in range(epochs):
            epoch_loss = 0
            for batch_users, batch_items, batch_ratings in train_loader:
                batch_users = batch_users.to(self.device)
                batch_items = batch_items.to(self.device)
                batch_ratings = batch_ratings.to(self.device)
                
                optimizer.zero_grad()
                
                # Forward pass
                predictions = model(batch_users, batch_items)
                
                # Calculate loss with L2 regularization
                mse_loss = criterion(predictions, batch_ratings)
                l2_loss = model.get_l2_loss()
                total_loss = mse_loss + l2_loss
                
                # Backward pass
                total_loss.backward()
                optimizer.step()
                
                epoch_loss += total_loss.item()
            
            avg_epoch_loss = epoch_loss / len(train_loader)
            train_losses.append(avg_epoch_loss)
            
            # Print progress every 10 epochs
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_epoch_loss:.4f}")
        
        # Evaluation
        model.eval()
        all_predictions = []
        all_targets = []
        
        with torch.no_grad():
            for batch_users, batch_items, batch_ratings in test_loader:
                batch_users = batch_users.to(self.device)
                batch_items = batch_items.to(self.device)
                batch_ratings = batch_ratings.to(self.device)
                
                predictions = model(batch_users, batch_items)
                
                all_predictions.extend(predictions.cpu().numpy())
                all_targets.extend(batch_ratings.cpu().numpy())
        
        # Calculate metrics
        all_predictions = np.array(all_predictions)
        all_targets = np.array(all_targets)
        
        test_rmse = np.sqrt(np.mean((all_targets - all_predictions)**2))
        test_mae = np.mean(np.abs(all_targets - all_predictions))
        
        print(f"NCF Results - RMSE: {test_rmse:.4f}, MAE: {test_mae:.4f}")
        
        return model, test_rmse, test_mae
    
    def ensemble_methods(self):
        """
        Ensemble of multiple collaborative filtering models
        """
        print("\n=== ENSEMBLE METHODS ===")
        
        # Prepare Surprise data
        reader = Reader(rating_scale=self.rating_scale)
        data = SurpriseDataset.load_from_df(self.df[['user_id', 'item_id', 'rating']], reader)
        trainset, testset = surprise_split(data, test_size=0.2, random_state=42)
        
        # Individual models with different hyperparameters
        models = {
            'SVD1': SVD(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02),
            'SVD2': SVD(n_factors=100, n_epochs=30, lr_all=0.01, reg_all=0.05),
            'SVD3': SVD(n_factors=150, n_epochs=40, lr_all=0.007, reg_all=0.03),
            'NMF1': NMF(n_factors=50, n_epochs=30, reg_pu=0.06, reg_qi=0.06),
            'NMF2': NMF(n_factors=100, n_epochs=40, reg_pu=0.1, reg_qi=0.1),
            'KNN': KNNWithMeans(k=40, sim_options={'name': 'pearson', 'user_based': False})
        }
        
        # Train all models
        trained_models = {}
        individual_predictions = {}
        
        for name, model in models.items():
            print(f"Training {name}...")
            model.fit(trainset)
            predictions = model.test(testset)
            
            # Calculate individual performance
            rmse_score = rmse(predictions, verbose=False)
            mae_score = mae(predictions, verbose=False)
            print(f"{name}: RMSE={rmse_score:.4f}, MAE={mae_score:.4f}")
            
            trained_models[name] = model
            individual_predictions[name] = predictions
        
        # Create ensemble predictions
        print("\nCreating ensemble...")
        ensemble_predictions = []
        
        for i in range(len(testset)):
            uid = testset[i][0]
            iid = testset[i][1]
            true_rating = testset[i][2]
            
            # Get predictions from all models
            model_preds = []
            for name, model in trained_models.items():
                pred = model.predict(uid, iid)
                model_preds.append(pred.est)
            
            # Simple average ensemble
            ensemble_est = np.mean(model_preds)
            
            # Weighted ensemble (give more weight to better performing models)
            # You can adjust weights based on individual model performance
            weights = [0.2, 0.2, 0.15, 0.15, 0.15, 0.15]  # Adjust as needed
            weighted_ensemble_est = np.average(model_preds, weights=weights)
            
            from surprise import Prediction
            ensemble_predictions.append(Prediction(uid, iid, true_rating, weighted_ensemble_est, {}))
        
        # Evaluate ensemble
        ensemble_rmse = rmse(ensemble_predictions, verbose=False)
        ensemble_mae = mae(ensemble_predictions, verbose=False)
        
        print(f"\nEnsemble Results - RMSE: {ensemble_rmse:.4f}, MAE: {ensemble_mae:.4f}")
        
        return ensemble_predictions, ensemble_rmse, ensemble_mae
    
    def implicit_feedback_approach(self):
        """
        Implicit feedback approach using matrix factorization
        """
        print("\n=== IMPLICIT FEEDBACK APPROACH ===")
        
        # Create user-item interaction matrix
        interaction_matrix = self.df.pivot_table(
            index='user_encoded', 
            columns='item_encoded', 
            values='rating', 
            fill_value=0
        )
        
        print(f"Interaction matrix shape: {interaction_matrix.shape}")
        print(f"Sparsity: {(interaction_matrix == 0).sum().sum() / interaction_matrix.size * 100:.2f}%")
        
        # Simple implicit feedback with confidence weighting
        # Convert ratings to binary (0/1) and add confidence
        binary_matrix = (interaction_matrix > 0).astype(int)
        confidence_matrix = 1 + interaction_matrix * 10  # Higher rating = higher confidence
        
        # Use SVD with implicit feedback interpretation
        from surprise import SVD
        
        # Create implicit dataset
        implicit_data = []
        for user_idx in range(len(binary_matrix)):
            for item_idx in range(len(binary_matrix.columns)):
                if binary_matrix.iloc[user_idx, item_idx] == 1:
                    # Positive feedback
                    confidence = confidence_matrix.iloc[user_idx, item_idx]
                    # Add multiple entries based on confidence
                    for _ in range(int(confidence)):
                        implicit_data.append((user_idx, item_idx, 1))
                else:
                    # Negative feedback (sample some)
                    if np.random.random() < 0.1:  # Sample 10% of negative feedback
                        implicit_data.append((user_idx, item_idx, 0))
        
        # Convert to DataFrame
        implicit_df = pd.DataFrame(implicit_data, columns=['user_id', 'item_id', 'rating'])
        
        # Train implicit model
        reader = Reader(rating_scale=(0, 1))
        implicit_surprise_data = SurpriseDataset.load_from_df(implicit_df, reader)
        trainset, testset = surprise_split(implicit_surprise_data, test_size=0.2, random_state=42)
        
        implicit_model = SVD(n_factors=100, n_epochs=30, lr_all=0.01, reg_all=0.02)
        implicit_model.fit(trainset)
        
        predictions = implicit_model.test(testset)
        implicit_rmse = rmse(predictions, verbose=False)
        implicit_mae = mae(predictions, verbose=False)
        
        print(f"Implicit Feedback Results - RMSE: {implicit_rmse:.4f}, MAE: {implicit_mae:.4f}")
        
        return implicit_model, implicit_rmse, implicit_mae
    
    def advanced_regularization_svd(self):
        """
        SVD with advanced regularization techniques
        """
        print("\n=== ADVANCED REGULARIZATION SVD ===")
        
        reader = Reader(rating_scale=self.rating_scale)
        data = SurpriseDataset.load_from_df(self.df[['user_id', 'item_id', 'rating']], reader)
        trainset, testset = surprise_split(data, test_size=0.2, random_state=42)
        
        # Try different regularization strategies
        reg_models = {
            'High_Reg': SVD(n_factors=100, n_epochs=30, lr_all=0.005, reg_all=0.1),
            'Biased_Reg': SVD(n_factors=100, n_epochs=30, lr_all=0.01, reg_all=0.02, biased=True),
            'Low_LR': SVD(n_factors=150, n_epochs=50, lr_all=0.002, reg_all=0.05),
            'Adaptive': SVD(n_factors=100, n_epochs=30, lr_all=0.01, reg_all=0.02, biased=True)
        }
        
        results = {}
        for name, model in reg_models.items():
            model.fit(trainset)
            predictions = model.test(testset)
            rmse_score = rmse(predictions, verbose=False)
            mae_score = mae(predictions, verbose=False)
            results[name] = {'rmse': rmse_score, 'mae': mae_score, 'model': model}
            print(f"{name}: RMSE={rmse_score:.4f}, MAE={mae_score:.4f}")
        
        # Return best model
        best_model_name = min(results.keys(), key=lambda x: results[x]['rmse'])
        print(f"Best regularization approach: {best_model_name}")
        
        return results[best_model_name]['model'], results[best_model_name]['rmse'], results[best_model_name]['mae']
    
    def run_all_techniques(self):
        """
        Run all advanced techniques and compare results
        """
        print("=== RUNNING ALL ADVANCED TECHNIQUES ===")
        
        all_results = {}
        
        # 1. Neural Collaborative Filtering (PyTorch)
        try:
            ncf_model, ncf_rmse, ncf_mae = self.neural_collaborative_filtering()
            all_results['NCF_PyTorch'] = {'rmse': ncf_rmse, 'mae': ncf_mae}
        except Exception as e:
            print(f"NCF failed: {e}")
            all_results['NCF_PyTorch'] = {'rmse': float('inf'), 'mae': float('inf')}
        
        # 2. Ensemble Methods
        try:
            _, ensemble_rmse, ensemble_mae = self.ensemble_methods()
            all_results['Ensemble'] = {'rmse': ensemble_rmse, 'mae': ensemble_mae}
        except Exception as e:
            print(f"Ensemble failed: {e}")
            all_results['Ensemble'] = {'rmse': float('inf'), 'mae': float('inf')}
        
        # 3. Implicit Feedback
        try:
            _, implicit_rmse, implicit_mae = self.implicit_feedback_approach()
            all_results['Implicit'] = {'rmse': implicit_rmse, 'mae': implicit_mae}
        except Exception as e:
            print(f"Implicit failed: {e}")
            all_results['Implicit'] = {'rmse': float('inf'), 'mae': float('inf')}
        
        # 4. Advanced Regularization
        try:
            _, reg_rmse, reg_mae = self.advanced_regularization_svd()
            all_results['Advanced_Reg'] = {'rmse': reg_rmse, 'mae': reg_mae}
        except Exception as e:
            print(f"Advanced Regularization failed: {e}")
            all_results['Advanced_Reg'] = {'rmse': float('inf'), 'mae': float('inf')}
        
        # Summary
        print("\n=== FINAL COMPARISON ===")
        print("Method\t\tRMSE\t\tMAE")
        print("-" * 40)
        for method, results in all_results.items():
            print(f"{method}\t\t{results['rmse']:.4f}\t\t{results['mae']:.4f}")
        
        # Best method
        best_method = min(all_results.keys(), key=lambda x: all_results[x]['rmse'])
        print(f"\nBest method: {best_method}")
        print(f"Best RMSE: {all_results[best_method]['rmse']:.4f}")
        print(f"Best MAE: {all_results[best_method]['mae']:.4f}")
        
        return all_results

# === USAGE EXAMPLE ===
"""
# Initialize with your dataframe
advanced_cf = AdvancedCollaborativeFiltering(df, rating_scale=(0, 1))

# Run all techniques
results = advanced_cf.run_all_techniques()

# Or run individual techniques
# ncf_model, ncf_rmse, ncf_mae = advanced_cf.neural_collaborative_filtering()
# ensemble_pred, ens_rmse, ens_mae = advanced_cf.ensemble_methods()
"""

'\n# Initialize with your dataframe\nadvanced_cf = AdvancedCollaborativeFiltering(df, rating_scale=(0, 1))\n\n# Run all techniques\nresults = advanced_cf.run_all_techniques()\n\n# Or run individual techniques\n# ncf_model, ncf_rmse, ncf_mae = advanced_cf.neural_collaborative_filtering()\n# ensemble_pred, ens_rmse, ens_mae = advanced_cf.ensemble_methods()\n'

In [29]:
advanced_cf = AdvancedCollaborativeFiltering(df, rating_scale=(0, 2))
results = advanced_cf.run_all_techniques()

Using device: mps
Prepared data: 1448797 ratings, 4179 users, 5000 items
=== RUNNING ALL ADVANCED TECHNIQUES ===

=== NEURAL COLLABORATIVE FILTERING (PyTorch) ===
NCF Model Architecture:
NeuralCollaborativeFiltering(
  (user_embedding): Embedding(4179, 50)
  (item_embedding): Embedding(5000, 50)
  (mlp): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=1, bias=True)
    (7): Sigmoid()
  )
)
Total parameters: 480199
Epoch 10/50, Loss: 0.2290
Epoch 20/50, Loss: 0.2290
Epoch 30/50, Loss: 0.2290
Epoch 40/50, Loss: 0.2290
Epoch 50/50, Loss: 0.2290
NCF Results - RMSE: 0.4741, MAE: 0.4398

=== ENSEMBLE METHODS ===
Training SVD1...
SVD1: RMSE=0.3749, MAE=0.2636
Training SVD2...
SVD2: RMSE=0.3746, MAE=0.2667
Training SVD3...
SVD3: RMSE=0.3748, MAE=0.2646
Tra

In [30]:
results

{'NCF_PyTorch': {'rmse': 0.47413218, 'mae': 0.43983847},
 'Ensemble': {'rmse': 0.3744485822597834, 'mae': 0.259090336856249},
 'Implicit': {'rmse': 0.2409351085272274, 'mae': 0.15628666822459183},
 'Advanced_Reg': {'rmse': 0.3739094035226511, 'mae': 0.2664917083882635}}

In [31]:
advanced_cf = AdvancedCollaborativeFiltering(df_clean, rating_scale=(0, 2))
results = advanced_cf.run_all_techniques()

Using device: mps
Prepared data: 1276645 ratings, 1823 users, 4993 items
=== RUNNING ALL ADVANCED TECHNIQUES ===

=== NEURAL COLLABORATIVE FILTERING (PyTorch) ===
NCF Model Architecture:
NeuralCollaborativeFiltering(
  (user_embedding): Embedding(1823, 50)
  (item_embedding): Embedding(4993, 50)
  (mlp): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=64, out_features=1, bias=True)
    (7): Sigmoid()
  )
)
Total parameters: 362049
Epoch 10/50, Loss: 0.2295
Epoch 20/50, Loss: 0.2295
Epoch 30/50, Loss: 0.2295
Epoch 40/50, Loss: 0.2295
Epoch 50/50, Loss: 0.2295
NCF Results - RMSE: 0.4736, MAE: 0.4401

=== ENSEMBLE METHODS ===
Training SVD1...
SVD1: RMSE=0.3754, MAE=0.2631
Training SVD2...
SVD2: RMSE=0.3751, MAE=0.2661
Training SVD3...
SVD3: RMSE=0.3753, MAE=0.2640
Tra

In [32]:
results

{'NCF_PyTorch': {'rmse': 0.47359955, 'mae': 0.4400797},
 'Ensemble': {'rmse': 0.37511636081441874, 'mae': 0.2586405027922102},
 'Implicit': {'rmse': 0.22305644997804586, 'mae': 0.13013373128362954},
 'Advanced_Reg': {'rmse': 0.3743759917590676, 'mae': 0.266138846491769}}

Precision@10 (Quality): This tells us, "Of the 10 users we recommended, what percentage were actually good recommendations (liked or matched)?"
</br>
Recall@10 (Coverage): This answers, "Of all the people a user would have liked, what percentage did we successfully find in our top 10 recommendations?"

   * Precision/Recall answer: "How good are my top-k rankings?"
   * RMSE/MAE answer: "How good is my model at predicting the exact rating a user would give?"
<br>

#### MAE: Mean Absolute Error

*The question it answers:* "On average, how far off was our prediction from the user's actual rating?"

#### RMSE: Root Mean Squared Error

*The question it answers:* "How far off are our predictions, but with a special penalty for being very wrong?"

In [37]:
from collections import defaultdict
from surprise import Dataset, Reader, SVD, KNNBasic, NMF


def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...]
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def precision_recall_at_k(predictions, k=10, threshold=1):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@k: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@k: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

def run_evaluation():
    df = pd.read_csv('assignment/activity.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Time-based split
    train_df = df[df['timestamp'] < '2023-04-01']
    test_df = df[df['timestamp'] >= '2023-04-01']

    reader = Reader(rating_scale=(0, 2))
    train_data = Dataset.load_from_df(train_df[['user_id', 'liked_user_id', 'like_type']], reader)
    test_data = Dataset.load_from_df(test_df[['user_id', 'liked_user_id', 'like_type']], reader)

    trainset = train_data.build_full_trainset()
    testset = test_data.build_full_trainset().build_testset()

    models = {
        "SVD": SVD(),
        "KNNBasic": KNNBasic(),
        "NMF": NMF()
    }

    results = {}

    for name, model in models.items():
        print(f"Evaluating {name}...")
        model.fit(trainset)
        predictions = model.test(testset)
        
        precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=1)

        avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
        avg_recall = sum(rec for rec in recalls.values()) / len(recalls)
        
        results[name] = {"precision": avg_precision, "recall": avg_recall}
        
        print(f"  Precision@10: {avg_precision:.4f}")
        print(f"  Recall@10: {avg_recall:.4f}")
        print()

    print("--- Final Results ---")
    for name, metrics in results.items():
        print(f"{name}: Precision@10={metrics['precision']:.4f}, Recall@10={metrics['recall']:.4f}")

In [38]:
run_evaluation()

Evaluating SVD...
  Precision@10: 0.9888
  Recall@10: 0.1295

Evaluating KNNBasic...
Computing the msd similarity matrix...
Done computing similarity matrix.
  Precision@10: 0.9875
  Recall@10: 0.1378

Evaluating NMF...
  Precision@10: 0.9985
  Recall@10: 0.1138

--- Final Results ---
SVD: Precision@10=0.9888, Recall@10=0.1295
KNNBasic: Precision@10=0.9875, Recall@10=0.1378
NMF: Precision@10=0.9985, Recall@10=0.1138


=== ADDITIONAL IMPROVEMENTS TO TRY ===
1. FEATURE ENGINEERING:
   - Add user demographics (age, gender, location)
   - Add item features (category, price, popularity)
   - Time-based features (season, day of week)

2. MATRIX FACTORIZATION VARIANTS:
   - Non-negative Matrix Factorization (NMF)
   - Probabilistic Matrix Factorization (PMF)
   - Bayesian Personalized Ranking (BPR)

3. DEEP LEARNING ALTERNATIVES:
   - Autoencoders for collaborative filtering
   - Variational Autoencoders (VAE)
   - Neural Matrix Factorization
   - Graph Neural Networks (GNNs)

4. EXTERNAL LIBRARIES:
   - Install 'implicit' library: pip install implicit
   - Install 'lightfm' library: pip install lightfm
   - Try Microsoft Recommenders toolkit
   - PyTorch Geometric for graph-based methods

5. EVALUATION IMPROVEMENTS:
   - Use ranking metrics (NDCG, MAP)
   - Cross-validation with temporal splits
   - A/B testing framework

6. PYTORCH OPTIMIZATIONS:
   - Use mixed precision training (torch.cuda.amp)
   - Implement learning rate scheduling
   - Add gradient clipping for stability
   - Use DataParallel for multi-GPU training